In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/EECS442

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1w_pSrzSOj3H2lluEXNwazzJe5p_Ak2xN/EECS442


In [2]:
!pip install torchsummary
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install pytorch-ignite itk lmdb einops mlflow pynrrd clearml
%matplotlib inline

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 37.3 MB/s eta 0:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchsummary import summary
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader, random_split

import os
import json
import shutil
import tempfile
import time

import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai import transforms
from monai.transforms import (
    AsDiscrete,
    Activations,
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.utils.enums import MetricReduction
from monai.networks.nets import SwinUNETR
from monai import data
from monai.data import decollate_batch
from functools import partial
from glob import glob
from monai.transforms import Compose, LoadImaged, ConvertToMultiChannelBasedOnBratsClassesd, NormalizeIntensityd
from monai.data import DataLoader, Dataset

import torch


print_config()

MONAI version: 1.4.dev2415
Numpy version: 1.25.2
Pytorch version: 2.2.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 189d1865c1b5b228b9d9e5e95ed40969eda7badc
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.5.0.post2
ITK version: 5.3.0
Nibabel version: 4.0.2
scikit-image version: 0.19.3
scipy version: 1.11.4
Pillow version: 9.4.0
Tensorboard version: 2.15.2
gdown version: 4.7.3
TorchVision version: 0.17.1+cu121
tqdm version: 4.66.2
lmdb version: 1.4.1
psutil version: 5.9.5
pandas version: 2.0.3
einops version: 0.7.0
transformers version: 4.38.2
mlflow version: 2.12.1
pynrrd version: 1.0.0
clearml version: 1.15.1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = np.where(self.count > 0, self.sum / self.count, self.sum)


def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

def load_checkpoint(filename, model):
    # Check if CUDA is available, and set the map_location accordingly
    map_location = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Load the checkpoint with the specified map_location
    checkpoint = torch.load(filename, map_location=map_location)

    # Apply the state_dict to the model
    model.load_state_dict(checkpoint['state_dict'])

    start_epoch = checkpoint.get('epoch', 0) + 1  # Default to 0 if not found
    best_acc = checkpoint.get('best_acc', 0.0)  # Default to 0.0 if not found
    print(f"Loaded checkpoint from epoch {checkpoint.get('epoch', 'Unknown')} with best accuracy {best_acc:.4f}")

    return start_epoch, best_acc

def get_loader(batch_size, data_dir, json_list, fold, roi):
    data_dir = data_dir
    datalist_json = json_list
    train_files, validation_files = datafold_read(datalist=datalist_json, basedir=data_dir, fold=fold)
    train_transform = transforms.Compose(
        [
            transforms.LoadImaged(keys=["image", "label"]),
            transforms.ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
            transforms.CropForegroundd(
                keys=["image", "label"],
                source_key="image",
                k_divisible=[roi[0], roi[1], roi[2]],
            ),
            transforms.RandSpatialCropd(
                keys=["image", "label"],
                roi_size=[roi[0], roi[1], roi[2]],
                random_size=False,
            ),
            transforms.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
            transforms.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1),
            transforms.RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2),
            transforms.NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
            transforms.RandScaleIntensityd(keys="image", factors=0.1, prob=1.0),
            transforms.RandShiftIntensityd(keys="image", offsets=0.1, prob=1.0),
        ]
    )
    val_transform = transforms.Compose(
        [
            transforms.LoadImaged(keys=["image", "label"]),
            transforms.ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
            transforms.NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
        ]
    )

    train_ds = data.Dataset(data=train_files, transform=train_transform)

    train_loader = data.DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
    )
    val_ds = data.Dataset(data=validation_files, transform=val_transform)
    val_loader = data.DataLoader(
        val_ds,
        batch_size=1,
        shuffle=False,
        num_workers=8,
        pin_memory=True,
    )

    return train_loader, val_loader

if torch.cuda.is_available():
    print("Using the GPU. You are good to go!")
    device = 'cuda'
else:
    print("Using the CPU. Overall speed may be slowed down")
    device = 'cpu'



Using the GPU. You are good to go!


In [5]:
data_dir = "./"
root_dir = "/content/drive/MyDrive/EECS442/Jovan_Kai_Swin_unetr_brats21_segmentation_3d/DONOTDELETE_Saved_Checkpoints_BRATS21"
k_fold = 3 #either 2-fold or 3-fold
fold = 1
# json_list = "./Jovan_Kai_Swin_unetr_brats21_segmentation_3d/brats20_training_" + str(k_fold) + "fold.json"
json_list = "./data/brats21_folds.json"
roi = (128, 128, 128)
batch_size = 1
sw_batch_size = 2
infer_overlap = 0.5
max_epochs = 100
train_loader, val_loader = get_loader(batch_size, data_dir, json_list, fold, roi)

model = SwinUNETR(
    img_size=roi,
    in_channels=4,
    out_channels=3,
    feature_size=36, #decreased feature_size (must be divisible by 12 or else errors)
    drop_rate=0.0,
    attn_drop_rate=0.0,
    dropout_path_rate=0.0,
    use_checkpoint=True,
).to(device)

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


In [6]:
import torch
from monai.metrics import DiceMetric, ConfusionMatrixMetric


def evaluate_and_plot(model, device, test_loader, slice_num=67):
    # Load model
    model.load_state_dict(torch.load(os.path.join(root_dir, "model.pt"))["state_dict"])
    model.to(device)
    model.eval()

    # Metrics setup
    dice_metric = DiceMetric(include_background=True, reduction='mean', get_not_nans=False, ignore_empty=True)
    sensitivity_metric = ConfusionMatrixMetric(include_background=True, metric_name='sensitivity')
    specificity_metric = ConfusionMatrixMetric(include_background=True, metric_name='specificity')
    precision_metric = ConfusionMatrixMetric(include_background=True, metric_name='precision')
    accuracy_metric = ConfusionMatrixMetric(include_background=True, metric_name='accuracy')

    # Prepare inferer with sliding window
    model_inferer = partial(
        sliding_window_inference,
        roi_size=roi,
        sw_batch_size=1,
        predictor=model,
        overlap=0.6
    )

    # sensitivity_scores = []
    # specificity_scores = []
    # precision_scores = []
    # accuracy_scores = []
    epoch = 0
    with torch.no_grad():
        for batch_data in test_loader:
            print(f"-----EPOCH {epoch}-----")
            inputs = batch_data["image"].to(device)
            labels = batch_data["label"].to(device)
            outputs = model_inferer(inputs)

            # Update All Metric
            dice_metric(y_pred=outputs, y=labels)
            sensitivity_metric(y_pred=outputs, y=labels)
            specificity_metric(y_pred=outputs, y=labels)
            precision_metric(y_pred=outputs, y=labels)
            accuracy_metric(y_pred=outputs, y=labels)

            # Convert probabilities to binary output for visualization
            seg = torch.sigmoid(outputs)[0].detach().cpu().numpy()
            seg = (seg > 0.5).astype(np.int8)
            seg_out = np.zeros((seg.shape[1], seg.shape[2], seg.shape[3]))
            seg_out[seg[1] == 1] = 2
            seg_out[seg[0] == 1] = 1
            seg_out[seg[2] == 1] = 4

            # Load original image and label for visualization
            img = inputs[0][0].detach().cpu().numpy()  # Select the first channel for visualization
            lbl = labels[0][0].detach().cpu().numpy()  # Adjust similarly if labels have a channel dimension

            # Normalize img for better visualization if necessary
            img = (img - img.min()) / (img.max() - img.min())

            plt.figure("Evaluation", (18, 6))
            plt.subplot(1, 3, 1)
            plt.title("Image")
            plt.imshow(img[:, :, slice_num], cmap="gray")
            plt.subplot(1, 3, 2)
            plt.title("Label")
            plt.imshow(lbl[:, :, slice_num])
            plt.subplot(1, 3, 3)
            plt.title("Segmentation")
            plt.imshow(seg_out[:, :, slice_num])
            plt.show()
            epoch += 1


    # Aggregate results
    sensitivity = sensitivity_metric.aggregate()[0].item()
    specificity = specificity_metric.aggregate()[0].item()
    precision = precision_metric.aggregate()[0].item()
    accuracy = accuracy_metric.aggregate()[0].item()
    mean_dice = dice_metric.aggregate()[0].item()

    sensitivity_metric.reset()
    specificity_metric.reset()
    precision_metric.reset()
    accuracy_metric.reset()
    dice_metric.reset()

    return {
        'mean_dice': mean_dice,
        'mean_sensitivity': sensitivity,
        'mean_specificity': specificity,
        'mean_precision': precision,
        'mean_accuracy': accuracy
    }

In [7]:
%cd /content/drive/MyDrive/EECS442
data_dir = "./data/"

def generate_test_files():
    test_files = []
    for case_num in range(21, 53):

        # To evaluate model trained on BRATS2021
        image_base_path = os.path.join(data_dir, f"MICCAI_BraTS2020_TrainingData/BraTS20_Training_3{case_num}")
        label_path = os.path.join(image_base_path, f"BraTS20_Training_3{case_num}_seg.nii.gz")

        # Construct file paths
        image_paths = [
            os.path.join(image_base_path, f"BraTS20_Training_3{case_num}_flair.nii.gz"),
            os.path.join(image_base_path, f"BraTS20_Training_3{case_num}_t1ce.nii.gz"),
            os.path.join(image_base_path, f"BraTS20_Training_3{case_num}_t1.nii.gz"),
            os.path.join(image_base_path, f"BraTS20_Training_3{case_num}_t2.nii.gz"),
        ]

        # Check if all image files and label file exist and are not empty
        if all(os.path.exists(path) and os.path.getsize(path) > 0 for path in image_paths) and os.path.exists(label_path) and os.path.getsize(label_path) > 0:
            case_dict = {
                "image": image_paths,
                "label": label_path,
            }
            test_files.append(case_dict)
        else:
            print(f"Missing or empty files for case number {case_num}")

        # To evaluate model trained on BRATS2020
        # img = os.path.join(data_dir, f"TrainingData/BraTS2021_000{case_num}/BraTS2021_000{case_num}_flair.nii.gz")
        # label = os.path.join(data_dir, f"TrainingData/BraTS2021_000{case_num}/BraTS2021_000{case_num}_seg.nii.gz")
        # if os.path.exists(img) and os.path.exists(label):
        #   case_dict = {
        #       "image": [
        #           os.path.join(data_dir, f"TrainingData/BraTS2021_000{case_num}/BraTS2021_000{case_num}_flair.nii.gz"),
        #           os.path.join(data_dir, f"TrainingData/BraTS2021_000{case_num}/BraTS2021_000{case_num}_t1ce.nii.gz"),
        #           os.path.join(data_dir, f"TrainingData/BraTS2021_000{case_num}/BraTS2021_000{case_num}_t1.nii.gz"),
        #           os.path.join(data_dir, f"TrainingData/BraTS2021_000{case_num}/BraTS2021_000{case_num}_t2.nii.gz"),
        #       ],
        #       "label": os.path.join(data_dir, f"TrainingData/BraTS2021_000{case_num}/BraTS2021_000{case_num}_seg.nii.gz"),

        #   }
        #   test_files.append(case_dict)
        case_num += 1
    return test_files

test_files = generate_test_files()

test_transform = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
        NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    ]
)

test_ds = Dataset(data=test_files, transform=test_transform)

test_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
results = evaluate_and_plot(model, device, test_loader)
print("Evaluation Results:", results)

Output hidden; open in https://colab.research.google.com to view.